# Description  
[Colab Link](https://colab.research.google.com/drive/1HSaMX9KLs2a0y3unixrTbHPSH9uKs4Y3?usp=sharing)  
Одной из самых больших проблем при покупке подержанного автомобиля на автоаукционе является риск того, что у машины могут быть серьезные проблемы, которые не позволят продать ее клиентам. В автомобильном сообществе такие неудачные покупки называют "киками".

Кикнутые автомобили часто становятся результатом подделки одометров, механических проблем, которые дилер не в состоянии решить, проблем с получением права собственности на автомобиль от продавца или других непредвиденных проблем. Кикнутые автомобили могут обойтись дилерам очень дорого, если учесть транспортные расходы, затраты на ремонт и рыночные потери при перепродаже автомобиля.

Специалисты по моделированию, способные определить, какие автомобили имеют более высокий риск стать "киком", могут принести реальную пользу дилерским центрам, стремящимся предоставить своим клиентам наилучший выбор товарного запаса.

Задача этого конкурса - предсказать, является ли автомобиль, купленный на аукционе, "киком" (неудачной покупкой).  
- **Target:** `'IsBadBuy'`

Field Name		---		Definition
- RefID				---        Unique (sequential) number assigned to vehicles
- IsBadBuy	(target)		--- 	Identifies if the kicked vehicle was an avoidable - purchase
- PurchDate			--- 	The Date the vehicle was Purchased at Auction
- Auction			--- 		Auction provider at which the  vehicle was purchased
- VehYear			--- 		The manufacturer's year of the vehicle
- VehicleAge		--- 		The Years elapsed since the manufacturer's year
- Make				--- 	Vehicle Manufacturer
- Model				--- 	Vehicle Model
- Trim				--- 	Vehicle Trim Level
- SubModel			--- 	Vehicle Submodel
- Color				--- 	Vehicle Color
- Transmission		--- 		Vehicles transmission type (Automatic, Manual)
- WheelTypeID		--- 		The type id of the vehicle wheel
- WheelType			--- 	The vehicle wheel type description (Alloy, Covers)
- VehOdo			--- 		The vehicles odometer reading
- Nationality		--- 		The Manufacturer's country
- Size				--- 	The size category of the vehicle (Compact, SUV, etc.)
- TopThreeAmericanName		--- 	Identifies if the manufacturer is one of the top three American manufacturers
- MMRAcquisitionAuctionAveragePrice	--- Acquisition price for this vehicle in average condition at time of purchase
- MMRAcquisitionAuctionCleanPrice	--- 	Acquisition price for this vehicle in the above Average condition at time of purchase
- MMRAcquisitionRetailAveragePrice	--- Acquisition price for this vehicle in the retail market in average condition at time of purchase
- MMRAcquisitonRetailCleanPrice		--- Acquisition price for this vehicle in the retail market in above average condition at time of purchase
- MMRCurrentAuctionAveragePrice		--- Acquisition price for this vehicle in average condition as of current day
- MMRCurrentAuctionCleanPrice		--- Acquisition price for this vehicle in the above condition as of current day
- MMRCurrentRetailAveragePrice		--- Acquisition price for this vehicle in the retail market in average condition as of current day
- MMRCurrentRetailCleanPrice		--- Acquisition price for this vehicle in the retail market in above average condition as of current day
- PRIMEUNIT				--- Identifies if the vehicle would have a higher demand than a standard purchase
- AcquisitionType				--- Identifies how the vehicle was aquired (Auction buy, trade in, etc)
- AUCGUART				--- The level guarntee provided by auction for the vehicle (Green light - Guaranteed/arbitratable, Yellow Light - caution/issue, red light - sold as is)
- KickDate				--- Date the vehicle was kicked back to the auction
- BYRNO					--- Unique number assigned to the buyer that purchased the vehicle
- VNZIP                ---                    Zipcode where the car was purchased
- VNST                ---                     State where the the car was purchased
- VehBCost				--- Acquisition cost paid for the vehicle at time of purchase
- IsOnlineSale		--- 		Identifies if the vehicle was originally purchased online
- WarrantyCost       ---                      Warranty price (term=36month  and millage=36K)





In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, roc_auc_score, precision_recall_curve, mean_squared_error
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from tqdm import tqdm
import warnings
warnings. filterwarnings('ignore')

# 1. Data preparation

## 1. Download data from Don’tGetKicked competition.

In [3]:
df = pd.read_csv('data/training.csv')
df.head(5)

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,3,0,12/7/2009,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,...,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389
3,4,0,12/7/2009,ADESA,2004,5,DODGE,NEON,SXT,4D SEDAN,...,4375.0,5518.0,NaN,NaN,19638,33619,FL,4100.0,0,630
4,5,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,...,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   RefId                              72983 non-null  int64  
 1   IsBadBuy                           72983 non-null  int64  
 2   PurchDate                          72983 non-null  object 
 3   Auction                            72983 non-null  object 
 4   VehYear                            72983 non-null  int64  
 5   VehicleAge                         72983 non-null  int64  
 6   Make                               72983 non-null  object 
 7   Model                              72983 non-null  object 
 8   Trim                               70623 non-null  object 
 9   SubModel                           72975 non-null  object 
 10  Color                              72975 non-null  object 
 11  Transmission                       72974 non-null  obj

In [8]:
df.IsBadBuy.value_counts()

0    64007
1     8976
Name: IsBadBuy, dtype: int64

## 2. Design train/validation/test split.
Use “PurchDate” field for splitting, test must be later in time than validation, the same goes for validation and train: train.PurchDate < valid.PurchDate < test.PurchDate.
Use the first 33% of dates for train, last 33% of dates for test, and middle 33% for validation set.
Don’t use the test dataset until the end!

In [9]:
df_prepared = df.copy()

проверим дубликаты пропуски

In [10]:
df_prepared.duplicated().value_counts()

False    72983
dtype: int64

In [11]:
def print_useful_rows_info(df):
    """Количество и процент заполненных строк"""
    print('Amount of useful rows:', len(df.dropna()))
    print('Persentage of filled rows', round(len(df.dropna()) / len(df) * 100, 2))

In [12]:
print_useful_rows_info(df_prepared)

Amount of useful rows: 3276
Persentage of filled rows 4.49


In [13]:
def blank_rows_percentage(df):
  """Вывод колонок и процента пропусков в каждой"""
  print((df.isna().sum() / len(df) * 100).sort_values(ascending=False))

In [14]:
blank_rows_percentage(df_prepared)

PRIMEUNIT                            95.315347
AUCGUART                             95.315347
WheelType                             4.348958
WheelTypeID                           4.342107
Trim                                  3.233630
MMRCurrentAuctionAveragePrice         0.431607
MMRCurrentRetailCleanPrice            0.431607
MMRCurrentRetailAveragePrice          0.431607
MMRCurrentAuctionCleanPrice           0.431607
MMRAcquisitionAuctionAveragePrice     0.024663
MMRAcquisitionAuctionCleanPrice       0.024663
MMRAcquisitionRetailAveragePrice      0.024663
MMRAcquisitonRetailCleanPrice         0.024663
Transmission                          0.012332
SubModel                              0.010961
Color                                 0.010961
Nationality                           0.006851
Size                                  0.006851
TopThreeAmericanName                  0.006851
BYRNO                                 0.000000
VNZIP1                                0.000000
VNST         

в столбцах  PRIMEUNIT  и AUCGUART больше 90% пропусков, удалим эти столбцы

In [15]:
df_prepared.drop(columns=['PRIMEUNIT', 'AUCGUART'], axis=0, inplace=True)

In [16]:
print_useful_rows_info(df_prepared)

Amount of useful rows: 67270
Persentage of filled rows 92.17


остальные пропуски заполним

In [17]:
df_prepared['WheelType'].value_counts()

Alloy      36050
Covers     33004
Special      755
Name: WheelType, dtype: int64

In [18]:
df_prepared['WheelType'].isna().sum()

3174

In [19]:
df_prepared[df_prepared['WheelType'].isna()]['WheelTypeID'].isna().sum()

3169

In [20]:
df_prepared['WheelTypeID'].value_counts()

1.0    36050
2.0    33004
3.0      755
0.0        5
Name: WheelTypeID, dtype: int64

чтобы сохранить больше данных, заполним ячейки как 'other'

In [21]:
df_prepared['WheelType'] = df_prepared['WheelType'].fillna('other')
df_prepared['WheelTypeID'] = df_prepared['WheelTypeID'].fillna(-1.0)

In [22]:
df_prepared['Trim'].value_counts()

Bas    13950
LS     10174
SE      9348
SXT     3825
LT      3540
       ...  
Har        1
LL         1
JLX        1
JLS        1
L 3        1
Name: Trim, Length: 134, dtype: int64

In [23]:
df_prepared['Trim'] = df_prepared['Trim'].fillna('other')

In [24]:
print_useful_rows_info(df_prepared)

Amount of useful rows: 72658
Persentage of filled rows 99.55


In [25]:
df_prepared[df_prepared.isnull().any(axis=1)]['IsBadBuy'].value_counts(normalize=True)

0    0.898462
1    0.101538
Name: IsBadBuy, dtype: float64

In [26]:
df_prepared[df_prepared.notnull()]['IsBadBuy'].value_counts(normalize=True)

0    0.877012
1    0.122988
Name: IsBadBuy, dtype: float64

остальных пропусков меньше 0,5% и они имеют тот же дисбаланс классов целевой переменной, поэтому
можно их удалить

In [27]:
df_prepared.dropna(inplace=True)

преобразуем категориальные переменные

удалим разные айдишники

In [28]:
df_prepared.drop(columns=['RefId', 'BYRNO'], axis=1, inplace=True)

преобразуем категориальные переменные

In [29]:
df_prepared['Model'] = df_prepared['Model'].astype('category')
df_prepared['SubModel'] = df_prepared['SubModel'].astype('category')
df_prepared['Auction'] = df_prepared['Auction'].astype('category')
df_prepared['Make'] = df_prepared['Make'].astype('category')
df_prepared['Trim'] = df_prepared['Trim'].astype('category')
df_prepared['Color'] = df_prepared['Color'].astype('category')
df_prepared['Transmission'] = df_prepared['Transmission'].astype('category')
df_prepared['Nationality'] = df_prepared['Nationality'].astype('category')
df_prepared['VNST'] = df_prepared['VNST'].astype('category')
df_prepared['VNZIP1'] = df_prepared['VNZIP1'].astype('category')
df_prepared['WheelType'] = df_prepared['WheelType'].astype('category')
df_prepared['Size'] = df_prepared['Size'].astype('category')
df_prepared['TopThreeAmericanName'] = df_prepared['TopThreeAmericanName'].astype('category')

In [30]:
df_prepared['PurchDate'].dtype

dtype('O')

Как видим 'PurchDate' является типом объект, преобразуем в дату

In [31]:
df_prepared['PurchDate'] = pd.to_datetime(df_prepared['PurchDate'], errors='coerce') #ставим NaT в случае ошибки

if df_prepared['PurchDate'].notnull().all():
    print("All dates are valid.")
else:
    print("Some dates are invalid.")

All dates are valid.


In [32]:
df_prepared.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72658 entries, 0 to 72982
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   IsBadBuy                           72658 non-null  int64         
 1   PurchDate                          72658 non-null  datetime64[ns]
 2   Auction                            72658 non-null  category      
 3   VehYear                            72658 non-null  int64         
 4   VehicleAge                         72658 non-null  int64         
 5   Make                               72658 non-null  category      
 6   Model                              72658 non-null  category      
 7   Trim                               72658 non-null  category      
 8   SubModel                           72658 non-null  category      
 9   Color                              72658 non-null  category      
 10  Transmission                      

In [33]:
df_prepared.sort_values(by='PurchDate', inplace=True)

In [34]:
train_val, test = train_test_split(df_prepared, test_size=0.33, shuffle=False)
train, val = train_test_split(train_val, test_size=0.5, shuffle=False)

print(f"train date from {train['PurchDate'].min()} to {train['PurchDate'].max()} size:{train.shape[0]}")
print(f"val   date from {val['PurchDate'].min()} to {val['PurchDate'].max()} size:{val.shape[0]}")
print(f"test  date from {test['PurchDate'].min()} to {test['PurchDate'].max()} size:{test.shape[0]}")

train date from 2009-01-05 00:00:00 to 2009-09-16 00:00:00 size:24340
val   date from 2009-09-16 00:00:00 to 2010-05-18 00:00:00 size:24340
test  date from 2010-05-18 00:00:00 to 2010-12-30 00:00:00 size:23978


## 3. Preprocess categorical variables
Use LabelEncoder or OneHotEncoder from sklearn to preprocess categorical variables. Be careful with data leakage (fit Encoder on train and apply on validation & test). Consider another encoding approach if you meet new categorical values in valid & test (unseen in the training dataset), for example: https://contrib.scikit-learn.org/category_encoders/count.html

In [35]:
X_train = train.drop(columns=['IsBadBuy', 'PurchDate'], axis=1)
y_train = train['IsBadBuy']

X_val = val.drop(columns=['IsBadBuy', 'PurchDate'], axis=1)
y_val = val['IsBadBuy']

X_test = test.drop(columns=['IsBadBuy', 'PurchDate'], axis=1)
y_test = test['IsBadBuy']

**For hot encoder:**  
Model  
SubModel  
Auction  
Make  
Trim  
Color  
Transmission  
Nationality  
VNST  
VNZIP1  

In [36]:
list_ohe_features = ['Model',
                    'SubModel',
                    'Auction',
                    'Make',
                    'Trim',
                    'Color',
                    'Transmission',
                    'Nationality',
                    'VNST',
                    'VNZIP1']
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

encoded = encoder.fit(X_train[list_ohe_features])

X_train_ohe = pd.DataFrame(encoder.transform(X_train[list_ohe_features]), columns=encoded.get_feature_names_out())
X_train = pd.concat([X_train.reset_index(), X_train_ohe.reset_index()], axis=1).drop(['index'], axis=1)
X_train.drop(list_ohe_features, axis=1, inplace=True)

X_val_ohe = pd.DataFrame(encoder.transform(X_val[list_ohe_features]), columns=encoded.get_feature_names_out())
X_val = pd.concat([X_val.reset_index(), X_val_ohe.reset_index()], axis=1).drop(['index'], axis=1)
X_val.drop(list_ohe_features, axis=1, inplace=True)

X_test_ohe = pd.DataFrame(encoder.transform(X_test[list_ohe_features]), columns=encoded.get_feature_names_out())
X_test = pd.concat([X_test.reset_index(), X_test_ohe.reset_index()], axis=1).drop(['index'], axis=1)
X_test.drop(list_ohe_features, axis=1, inplace=True)

In [37]:
X_train.shape

(24340, 1809)

**For label encoder:**  
WheelTypeID  
WheelType  
Size  
TopThreeAmericanName  

In [38]:
X_test[['WheelType', 'Size', 'TopThreeAmericanName']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23978 entries, 0 to 23977
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   WheelType             23978 non-null  category
 1   Size                  23978 non-null  category
 2   TopThreeAmericanName  23978 non-null  category
dtypes: category(3)
memory usage: 71.2 KB


In [39]:
list_le_features = ['WheelType',
                    'Size',
                    'TopThreeAmericanName']
le = LabelEncoder()

for f in list_le_features:
    le.fit(X_train[f])

    X_train[f] = pd.DataFrame(le.transform(X_train[f]))
    X_val[f] = pd.DataFrame(le.transform(X_val[f]))
    X_test[f] = pd.DataFrame(le.transform(X_test[f]))

In [40]:
X_train['TopThreeAmericanName'].value_counts()

2    8468
0    8090
1    4600
3    3182
Name: TopThreeAmericanName, dtype: int64

In [41]:
X_train.shape

(24340, 1809)

In [42]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24340 entries, 0 to 24339
Columns: 1809 entries, VehYear to VNZIP1_99224
dtypes: float64(1801), int64(8)
memory usage: 335.9 MB


# 2. Create a Python class for Decision Tree Classifier and Decision Tree Regressor (MSE loss).
It should support fit, predict_proba and predict methods. Also, maximal depth (max_depth) must be a parameter of your class. Use Gini impurity criterion as criterion for choice of splitting.
Here is the blueprint:  
`model = DecisionTreeClassifier(max_depth=7)`  
`model.fit(Xtrain, ytrain)`  
`model.predict_proba(Xvalid)`

- Create a separate class for Node. It should be able to contain data (sample features and targets), compute Gini impurity and have pointers to children (left and right nodes). For Regressor use standard deviation instead of Gini impurity.
- Implement a function that finds the best possible split in the current node.
- Combine previous steps into your working Decision Tree Classifier.
- *Implement extra-randomized tree via designing another function for best-split search.

In [43]:
tree = DecisionTreeClassifier(max_depth=5)
tree.fit(X_train.iloc[:, :300], y_train)
accuracy_score(y_val, tree.predict(X_val.iloc[:, :300]))

0.8835250616269515

In [44]:
class Node:
    def __init__(self, data=None, feature_idx=None, threshold=None, left=None, right=None, info_gain=None, var_red=None, leaf_ter=None, target=None) -> None:
        self.feature_idx = feature_idx
        self.threshold = threshold
        self.data = data
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.var_red = var_red
        self.leaf_ter = leaf_ter
        self.target = target

    def gini_impurity(self):
        data_i = self._calc_impurity(self.data)
        left_i = self._calc_impurity(self.left)
        right_i = self._calc_impurity(self.right)
        w_l = len(self.left) / len(self.data)
        w_r = len(self.right) / len(self.data)
        return data_i - (w_l*left_i + w_r*right_i)

    def _calc_impurity(self, y):
        class_lables = np.unique(y)
        gini = 0

        for lable in class_lables:
            p_lable = len(y[y==lable]) / len(y)
            gini += p_lable**2
        return 1 - gini

    def var_reduction(self):
        w_l = len(self.left) / len(self.data)
        w_r = len(self.right) / len(self.data)
        return np.var(self.data) - (w_l*np.var(self.left) + w_r*np.var(self.right))


In [65]:
class MyDTC():
    def __init__(self, max_depth = 2) -> None:
        self.max_depth = max_depth
        self.tree = None


    def fit(self, X: np.array, y: np.array) -> None:
        data = np.concatenate((X, y), axis=1)

        self.tree = self._build_tree(data)


    def predict(self, X: np.array) -> np.array:
        preds = [self._make_pred(self.tree, x) for x in X]

        return preds


    def _build_tree(self, data: np.array, curr_depth = 0):
        X, y = data[:, :-1], data[:,-1]
        n_samples, n_features = X.shape

        if n_samples > 0 and curr_depth < self.max_depth:
            best_split = self._make_best_split(data, n_samples, n_features)

            if 'max_ig' in best_split and best_split['max_ig'] > 0:
                tree_l = self._build_tree(best_split['best_data_l'], curr_depth+1)
                tree_r = self._build_tree(best_split['best_data_r'], curr_depth+1)

                return Node(feature_idx=best_split['best_f_idx'], left=tree_l, right=tree_r, target=y,
                            info_gain=best_split['max_ig'], threshold=best_split['best_thr'])

        leaf_value = max(list(y), key=list(y).count)
        return Node(leaf_ter=leaf_value, target=y)


    def _make_best_split(self, data, n_samples, n_features):
        best_split = {}
        max_ig = -float('inf')

        for f_idx in tqdm(range(n_features)):
            f_vals = data[:f_idx]
            possible_thresholds = np.unique(f_vals)

            for thr in possible_thresholds:
                data_l, data_r = self._split(data, f_idx, thr)
                if len(data_l) > 0 and len(data_r) > 0:
                    y, y_l, y_r = data[:,-1], data_l[:,-1],data_r[:,-1]
                    curr_ig = Node(data=y, left=y_l, right=y_r).gini_impurity()

                    if curr_ig > max_ig:
                        best_split['best_f_idx'] = f_idx
                        best_split['best_thr'] = thr
                        best_split['best_data_l'] = data_l
                        best_split['best_data_r'] = data_r
                        best_split['max_ig'] = curr_ig
                        max_ig = curr_ig

        return best_split



    def _split(self, data, f_idx, thr):
        data_l = np.array([r for r in data if r[f_idx] <= thr])
        data_r = np.array([r for r in data if r[f_idx] > thr])

        return data_l, data_r


    def _make_pred(self, node, x):
        if node.leaf_ter != None:
            return node.leaf_ter
        feature_value = x[node.feature_idx]
        if feature_value <= node.threshold:
            return self._make_pred(node.left, x)
        else:
            return self._make_pred(node.right, x)


    def predict_proba(self, X: np.array) -> np.array:
        probabilities = np.zeros((X.shape[0], len(np.unique(self.tree.target))))
        for i, x in enumerate(X):
            probabilities[i] = self._predict_proba(self.tree, x)
        return probabilities

    def _predict_proba(self, node, x):
        if node.leaf_ter is not None:
            class_labels = np.unique(node.target)
            class_counts = np.bincount(node.target.astype(int))
            return class_counts / len(node.target)
        else:
            if x[node.feature_idx] <= node.threshold:
                return self._predict_proba(node.left, x)
            else:
                return self._predict_proba(node.right, x)


In [46]:
class MyDTR():
    def __init__(self, max_depth = 2) -> None:
        self.max_depth = max_depth
        self.tree = None


    def fit(self, X: np.array, y: np.array) -> None:
        data = np.concatenate((X, y), axis=1)

        self.tree = self._build_tree(data)


    def predict(self, X: np.array) -> np.array:
        preds = [self._make_pred(self.tree, x) for x in X]

        return preds


    def _build_tree(self, data: np.array, curr_depth = 0):
        X, y = data[:, :-1], data[:,-1]
        n_samples, n_features = X.shape

        if n_samples > 0 and curr_depth < self.max_depth:
            best_split = self._make_best_split(data, n_samples, n_features)

            if 'max_red' in best_split and best_split['max_red'] > 0:
                tree_l = self._build_tree(best_split['best_data_l'], curr_depth+1)
                tree_r = self._build_tree(best_split['best_data_r'], curr_depth+1)

                return Node(feature_idx=best_split['best_f_idx'], left=tree_l, right=tree_r, target=y,
                            var_red=best_split['max_red'], threshold=best_split['best_thr'])

        leaf_value = np.mean(y)
        return Node(leaf_ter=leaf_value, target=y)


    def _make_best_split(self, data, n_samples, n_features):
        best_split = {}
        max_red = -float('inf')

        for f_idx in tqdm(range(n_features)):
            f_vals = data[:f_idx]
            possible_thresholds = np.unique(f_vals)

            for thr in possible_thresholds:
                data_l, data_r = self._split(data, f_idx, thr)
                if len(data_l) > 0 and len(data_r) > 0:
                    y, y_l, y_r = data[:,-1], data_l[:,-1],data_r[:,-1]
                    curr_red = Node(data=y, left=y_l, right=y_r).var_reduction()

                    if curr_red > max_red:
                        best_split['best_f_idx'] = f_idx
                        best_split['best_thr'] = thr
                        best_split['best_data_l'] = data_l
                        best_split['best_data_r'] = data_r
                        best_split['max_red'] = curr_red
                        max_red = curr_red

        return best_split



    def _split(self, data, f_idx, thr):
        data_l = np.array([r for r in data if r[f_idx] <= thr])
        data_r = np.array([r for r in data if r[f_idx] > thr])

        return data_l, data_r


    def _make_pred(self, node, x):
        if node.leaf_ter != None:
            return node.leaf_ter
        feature_value = x[node.feature_idx]
        if feature_value <= node.threshold:
            return self._make_pred(node.left, x)
        else:
            return self._make_pred(node.right, x)


    def predict_proba(self, X: np.array) -> np.array:
        probabilities = np.zeros((X.shape[0], len(np.unique(self.tree.target))))
        for i, x in enumerate(X):
            probabilities[i] = self._predict_proba(self.tree, x)
        return probabilities

    def _predict_proba(self, node, x):
        if node.leaf_ter is not None:
            class_labels = np.unique(node.target)
            class_counts = np.bincount(node.target.astype(int))
            return class_counts / len(node.target)
        else:
            if x[node.feature_idx] <= node.threshold:
                return self._predict_proba(node.left, x)
            else:
                return self._predict_proba(node.right, x)

In [47]:
cl = MyDTR(max_depth=3)
cl.fit(X_train.iloc[:,:4].values, X_train.iloc[:,4].values.reshape(-1,1))

100%|██████████| 4/4 [00:00<00:00, 43.93it/s]


In [48]:
np.sqrt(mean_squared_error(X_val.iloc[:,4], cl.predict(X_val.iloc[:,:4].values)))

13253.895289942155

In [49]:
clf = DecisionTreeRegressor(max_depth=3, criterion='squared_error')
clf.fit(X_train.iloc[:,:4], X_train.iloc[:,4])

DecisionTreeRegressor(max_depth=3)

In [50]:
np.sqrt(mean_squared_error(X_val.iloc[:,4], clf.predict(X_val.iloc[:,:4].values)))

13175.307700153746

In [51]:
X_train.iloc[:,:4]

,VehYear,VehicleAge,WheelTypeID,WheelType
0,2004,5,1.0,0
1,2004,5,1.0,0
2,2006,3,1.0,0
3,2005,4,1.0,0
4,2003,6,1.0,0
...,...,...,...,...
24335,2002,7,2.0,1
24336,2004,5,1.0,0
24337,2003,6,2.0,1
24338,2003,6,2.0,1


# 3. With your DecisionTree module you have to receive at least 0.1 Gini score on the validation dataset.

In [52]:
tr = MyDTC(max_depth=5)
tr.fit(X_train.iloc[:,:10].values, pd.DataFrame(y_train).values)

100%|██████████| 10/10 [00:00<00:00, 52.60it/s]


In [53]:
accuracy_score(y_val, tr.predict(X_val.iloc[:,:10].values))

0.8783894823336073

In [54]:
tr.predict_proba(X_val.iloc[:,:10].values)

array([[0.75465313, 0.24534687],
       [0.93829201, 0.06170799],
       [0.95499923, 0.04500077],
       ...,
       [0.95499923, 0.04500077],
       [0.95499923, 0.04500077],
       [0.95499923, 0.04500077]])

Gini score

In [55]:
2*roc_auc_score(y_val.to_numpy(), tr.predict(X_val.iloc[:,:10].values)) - 1

0.17003447455542364

# 4. Use DecisionTreeClassifier from sklearn and check its performance on validation dataset. Is it better than your module? If it is, why?

In [57]:
tree = DecisionTreeClassifier(max_depth=5, criterion='gini')
tree.fit(X_train.values[:,:10], y_train.values)

DecisionTreeClassifier(max_depth=5)

In [58]:
accuracy_score(y_val, tree.predict(X_val.iloc[:,:10].values))

0.878430566967954

In [59]:
tree.predict_proba(X_val.iloc[:,:10].values)

array([[0.75485009, 0.24514991],
       [0.94490255, 0.05509745],
       [0.95624345, 0.04375655],
       ...,
       [0.95624345, 0.04375655],
       [0.95624345, 0.04375655],
       [0.95624345, 0.04375655]])

Gini score

In [60]:
abs(2*roc_auc_score(y_val.to_numpy(), tree.predict(X_val.iloc[:,:10].values)) - 1)

0.16766864204467424

Лучше потому что больше настроек обучения есть а также наверно потому что на плюсах

# 5. Implement RandomForestClassifier and check its performance. You have to improve the result of one single tree and receive at least 0.15 Gini score on the validation dataset. Be able to set fixed random seed.

In [82]:
import random

class MyRFC:
    def __init__(self, n_estimators=10, max_depth=5, random_state=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        if self.random_state is not None:
            random.seed(self.random_state)
        self.trees = []

    def fit(self, X, y):
        for _ in range(self.n_estimators):
            indices = np.random.choice(len(X), size=len(X), replace=True)
            X_sample = X[indices]
            y_sample = y[indices]
            tree = MyDTC(max_depth=self.max_depth)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        predictions = []
        for sample in X:
            votes = []
            for tree in self.trees:
                votes.append(tree.predict([sample])[0])
            predictions.append(max(set(votes), key=votes.count))
        return np.array(predictions)


    def predict_proba(self, X):
        probabilities = np.array([tree.predict_proba(X) for tree in self.trees])
        return np.mean(probabilities, axis=0)


In [83]:
rfc = MyRFC(max_depth=2, n_estimators=3, random_state=21)
rfc.fit(X_train.iloc[:,:10].values, pd.DataFrame(y_train).values)

100%|██████████| 10/10 [00:12<00:00,  1.28s/it]


In [84]:
accuracy_score(y_val, rfc.predict(X_val.iloc[:,:10].values))

0.881922760887428

gini score

In [85]:
abs(2*roc_auc_score(y_val.to_numpy(), rfc.predict(X_val.iloc[:,:10].values)) - 1)

0.236032518052115

# 6. Using your DecisionTree design class for GDBT classifier.
This class must have max_depth, number_of_trees and max_features attributes. You have to compute the gradient of the binary cross entropy loss function and implement incremental learning: train the next tree using the results of previous trees.

# 7. Use LightGBM, Catboost and XGBoost to fit on a training set and prediction on validation dataset.
Check documentation of the libraries, and fine-tune the algorithms for the task.
Write down key differences between all implementations. Analyze special features of every algorithm (how do “categorical feature” work in Catboost, what is DART mode in XGBoost)?
What GBDT model gives the best result? Can you explain why?

In [93]:
!pip install lightgbm catboost xgboost -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 7.2 MB/s eta 0:00:00


In [94]:
import lightgbm as lgb
import catboost as cb
import xgboost as xgb

In [100]:
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.1,
    n_estimators=100,
    objective='binary',
    min_split_gain=0.01,
    min_child_samples=20,
    random_state=21
    )
lgb_model.fit(X_train, y_train)
lgb_preds = lgb_model.predict(X_val)

abs(2*roc_auc_score(y_val.to_numpy(), lgb_model.predict(X_val.values)) - 1)

0.23150229239697206

In [102]:
cb_params = {}
cb_model = cb.CatBoostClassifier(iterations=2, depth=10, learning_rate=0.1, loss_function='Logloss',
                              logging_level='Verbose')
cb_model.fit(X_train, y_train)
cb_preds = cb_model.predict(X_val)

abs(2*roc_auc_score(y_val.to_numpy(), cb_model.predict(X_val.values)) - 1)

0.2222792833019145

In [104]:
xgb_model = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=100,
 max_depth=10,
 min_child_weight=1,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_val)

abs(2*roc_auc_score(y_val.to_numpy(), xgb_model.predict(X_val.values)) - 1)


0.229516963088076

LightGBM:

Ключевые отличия: LightGBM использует алгоритм на основе гистограммы для поиска наилучшей точки разделения, что делает его более быстрым, чем другие алгоритмы повышения градиента. Он также поддерживает параллельное обучение и обучение на графическом процессоре.
Специальные возможности: LightGBM имеет встроенный механизм для обработки категориальных объектов. Он может автоматически обрабатывать категориальные объекты, не требуя явного кодирования. Он использует метод "односторонней выборки на основе градиента" (GOSS) для уменьшения использования памяти и повышения скорости обучения.  
  
Catboost:
Ключевые отличия: Catboost разработан для эффективной обработки категориальных функций. В нем используется инновационный алгоритм под названием "Упорядоченное повышение", который включает упорядочение категориальных переменных в процессе обучения. Он также поддерживает обучение на графическом процессоре.
Специальные возможности: Catboost автоматически обрабатывает категориальные функции по умолчанию. Он использует комбинацию однократного кодирования и алгоритм, называемый "Симметричными деревьями решений", для обработки категориальных переменных. Catboost также имеет встроенную поддержку для обработки пропущенных значений.
  
XGBoost:  
Ключевые отличия: XGBoost известен своей масштабируемостью и производительностью. Он поддерживает параллельные и распределенные вычисления и предоставляет различные методы регуляризации для предотвращения переобучения. Он также поддерживает обучение на графическом процессоре.
Специальные возможности: В XGBoost есть специальный режим под названием "DART" (Деревья аддитивной регрессии выпадения), который является улучшенной версией градиентного бустинга. DART вводит регуляризацию выпадения в процессе обучения, что помогает уменьшить переобучение. Он также имеет встроенную поддержку для обработки пропущенных значений.
Чтобы определить, какая модель GBDT дает наилучший результат, вам необходимо оценить их эффективность в наборе данных проверки, используя соответствующие оценочные показатели (например, точность, прецизионность, отзыв, F1-оценка). Наилучшей моделью будет та, которая обеспечивает наивысшую производительность на основе выбранного показателя.  

Выбор наилучшей модели может зависеть от различных факторов, таких как набор данных, характер проблемы и конкретные требования. Рекомендуется поэкспериментировать с различными моделями, настроить их гиперпараметры и оценить их производительность, чтобы принять обоснованное решение.

# 8. Pick up the best model and estimate its performance on the test dataset:
check Gini scores on all three datasets for your best model: train Gini, valid Gini, test Gini. Can you see any drop in performance when comparing valid quality vs test quality? Is your model overfitted or not?

In [105]:
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    num_leaves=31,
    max_depth=-1,
    learning_rate=0.1,
    n_estimators=100,
    objective='binary',
    min_split_gain=0.01,
    min_child_samples=20,
    random_state=21
    )
lgb_model.fit(X_train, y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2795, number of negative: 21545
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005838 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3986
[LightGBM] [Info] Number of data points in the train set: 24340, number of used features: 607
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.114832 -> initscore=-2.042312
[LightGBM] [Info] Start training from score -2.042312


LGBMClassifier(min_split_gain=0.01, objective='binary', random_state=21)

Train Gini

In [106]:
abs(2*roc_auc_score(y_train.to_numpy(), lgb_model.predict(X_train.values)) - 1)


0.26726687205835775

Val Gini

In [107]:
abs(2*roc_auc_score(y_val.to_numpy(), lgb_model.predict(X_val.values)) - 1)

0.23150229239697206

Test Gini

In [108]:
abs(2*roc_auc_score(y_test.to_numpy(), lgb_model.predict(X_test.values)) - 1)

0.22048498194425115

Просадка не больгая, думаю нормально обученная модель